In [2]:
from abc import ABC, abstractmethod
from datetime import datetime, date, timedelta

check valid params and store in the db

In [5]:
class PersonalHealth(ABC):
 
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id
        super().__init__()
    
    # Get the value corresponding to a particular column for a particular date
    @abstractmethod
    def get_value_given_date(self):
        pass
    
    # Get the daily nutrient/exercise? decomposition
    @abstractmethod
    def get_daily_decomposition(self):
        pass

    # Get the values corresponding to a particular column given a range of dates
    @abstractmethod
    def get_values_range(self):
        pass
    
    @abstractmethod
    def get_daily_value(self):
        pass
    
    @abstractmethod
    def get_weekly_values(self):
        pass
    
    @abstractmethod
    def get_monthly_values(self):
        pass
    
    @abstractmethod
    def get_from_API(self):
        pass
    
    @abstractmethod
    def insert_in_database(self):
        pass
    
    @abstractmethod
    def set_goals(self):
        pass
    
    @abstractmethod
    def check_goals(self):
        pass

Specify the default values

Check string for mysql

d = {'carbs': '"one"'}

Column Names for Diet Table in Database:
- 0) id (INT)
- 1) date_time (DATETIME)
- 2) item (STRING or VARCHAR(255))
- 3) servings (FLOAT)
- 4) upc (BOOL)
- 5) cals (FLOAT)
- 6) protein (FLOAT)
- 7) fat (FLOAT)
- 8) carbs (FLOAT)
- 9) fiber (FLOAT)

Column Names for Diet Table in Database:
- 0) id (INT)
- 1) date_time (DATETIME)
- 2) item (STRING or VARCHAR(255))
- 3) servings (INT)
- 4) upc (BOOL)
- 5) nutrient_type (STRING)

In [26]:
class Diet(PersonalHealth):
    
    def __init__(self, table_name, goal_table_name, database, user_id):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
        self.__user_id = user_id
    
    def get_value_given_date(self, nutrient, given_date):
        
        # Description: 
        # Takes a nutrient as input for instance protein and a date and return the
        # total amount proteins consumed on that date.
        
        # Input:
        # nutrient(STRING): 'cals, protein, fat, carbs, fiber'
        # given_date(DATE): python date, not datetime
        
        # Output:
        # Float value
        
        next_date = given_date + timedelta(days=1)
        
        query = f"SELECT servings, {nutrient} FROM {self.__table_name}\
                join Users on Users.id={self.__table_name}.{self.__user_id}\
                WHERE date_time >= '{str(given_date)} 00:00:00' AND\
                date_time <= '{str(next_date)} 00:00:00'"

        try:
            # If you can find the corresponding date in the database return the total 
            # nutrient consumed, otherwise just return 0
            records = self.__database.select_data(query)
            given_date_nutrients = 0
            for r in records:
                given_date_nutrients += r[0] * r[1]
            return given_date_nutrients
        except:
            return 0
    
    def get_daily_value(self, nutrient):
        
        # Description:
        # Useful for the main Diet page where we show total calories consumed.
        # For calories we will just call: get_daily_value('cal')
        
        # Input: 
        # nutrient (STRING): 'cals, protein, fat, carbs, fiber'
        
        # Output: 
        # Float value
        
        todays_date = date.today()
        return self.get_value_date(nutrient, todays_date)
    
    def get_daily_decomposition(self, nutrient_list): 
        
        # Description:
        # Useful for the diet page where we show the daily percentage breakup.
        # Look 2 diagram in Figure 1 in report.
        
        # Input:
        # nutrient_list (LIST of STRING): ['protein', 'fat', 'carbs', 'fiber']
        # There wont be cals because I think cals is a measure of energy and not a measure
        # of the mass, so we only have 4.
        
        # Output:
        # Returns a dictionary with keys as the nutrient_list and the corresponding percentage
        # as the values.
        
        nutri_decom = {}
        for nutri in nutrient_list:
            nutri_decom[nutri] = self.todays_value(nutri)
        
        total = sum([nutri_decom[k] for k in nutri_decom.keys()])
        
        per_nutri_decom = {}
        for nutri in nutrient_list:
            per_nutri_decom[nutri] = 100.0 * (nutri_decom[nutri]/total)
        
        return per_nutri_decom
    
    def get_values_range(self, nutrient, date, duration):
        
        # Description: 
        # Useful for making the weekly and monthly (if we are doing it) charts.
        # Just need to pass the start date and the duration, and it computes the 
        # values corresponding to a particular nutrient for that duration.
        
        # Input:
        # nutrient (STRING): 'cals, protein, fat, carbs, fiber'
        # date (DATE): python date, not datetime
        # duration (INT): the number of days before the given date. So for instance
        # for weekly date duration will be 7 and we will compute (date-7, date)
        
        # Output:
        # Returns a dictionary with the dates of that duration (date-duration, date) as the keys and the 
        # corresponding nutrient value as the values of the dictionary.
        
        nutrients_list = {}
        for i in range(duration):
            dt = date - timedelta(days=i)
            nutrients_list[str(dt)] = self.get_value_date(nutrient, dt)
        return nutrients_list
    
    def get_weekly_values(self, nutrient):
        
        # Description:
        # Weekly values for a particular nutrient.
        # Just call the previous function with duration = 7
        
        # Input:
        # nutrient (STRING): 'cals, protein, fat, carbs, fiber'
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last week and the corresponding 
        # nutrient values.
        
        return self.get_value_multiple_days(nutrient, date.today(), 7)
    
    def last_months_values(self, nutrient):
        
        # Description:
        # Sames as before just for a month.
        
        # Input:
        # nutrient (STRING): 'cals, protein, fat, carbs, fiber'
        
        # Output:
        # Returns a dictionary with the keys as the dates of the last month and the corresponding 
        # nutrient values.
        
        return self.get_value_multiple_days(nutrient, date.today(), 28)    
        
    def get_nutrient_from_API(self, query, upc):
        
        # Description: 
        # Calculates the nutrient values from the API
        
        # Input:
        # query (STRING): user input i.e. the string which user enters.
        # upc (BOOL): True if the query is a barcode number and False otherwise
        
        # Output:
        # label (STRING): the name of the food item in the database. The name which
        #                 the user enters may not be present in the database. We return the closest one.
        # api_dict (Dictionary): Dictionary with keys as the nutrient names: 'cals, protein, fat, carbs, fiber'
        #                        and values as the corresponding kcal or mg values in float.
        # success (BOOL): return True if an item with the query name is found in the API otherwise returns False.
        
        label, api_dict, success = self.get_nutrient_information(query, upc)
        return label, api_dict, success
    
    # Input dict should have the following keys:
    # 1)item
    # 2)servings
    # 3)upc
    # 4)manual_dict
    
    def insert_in_database(self, input_dict):
        # manual_dict: 
        nutrient_dict = manual_dict = input_dict['manual_dict']
        if manual_dict == None:
            label, api_dict, success = self.get_nutrient_from_API(input_dict['query'], input_dict['upc'])
            if not success:
                return False
            item = label
            nutrient_dict = api_dict
        
        data_dict = input_dict
        del data_dict['manual_dict']
        
        for k in nutrient_dict.keys():
            data_dict[k] = nutrient_dict[k]
            
        data_dict['date_time'] = str(datetime.now())

        self.__database.insert_Row(self.__table_name,data_dict)
        return True
    
    # Goal dict should have the following keys:
    # 1)calories
    # 2)protien
    # 3)carbohydrate
    # 4)fat
    # 5)fiber
    def set_goals(self, goal_dict):
        goal_dict['date_time'] = str(datetime.now())
        self.__database.insert_Row(self.__goal_table_name, goal_dict)
        
    def check_goal(self, nutrient):
        todays_date = date.today()
        todays_value = self.get_value_date(nutrient, todays_date)
        c = self.__database.get_Cursor()
        query = f"SELECT {nutrient} FROM {self.__goal_table_name}\
                ORDER BY ID DESC LIMIT 1"

        try:
            c.execute(query)
            records = c.fetchall()
            goal_value = records[0]
            if todays_value >= goal_value:
                return False
            else:
                return True
        except:
            False

Column Names for Fitness Table in Database:
- 0) user_id (INT)
- 1) date_time (DATETIME)
- 2) exercise_type (STRING or VARCHAR(255))
- 3) active_time (INT)
- 4) calories_burnt (INT)

In [26]:
class Fitness(PersonalHealth):
    
    def __init__(self, table_name, goal_table_name, database):
        self.__table_name = table_name
        self.__goal_table_name = goal_table_name
        self.__database = database
    
    def get_value_date(self, ex_type, given_date):
        next_date = given_date + timedelta(days=1)
        
        c = self.__database.get_Cursor()
        query = f"SELECT active_time FROM {self.__table_name}\
                WHERE date_time >= {str(given_date)} 00:00:00 AND\
                date_time <= {str(next_date)} 00:00:00 AND\ 
                exercise_type = {str(ex_type)}"

        try:
            c.execute(query)
            records = c.fetchall()

            total_active_time = 0
            for r in records:
                total_active_time += r
            return total_active_time
        except:
            return 1e-8

    def get_total_value_date(self, ex_list, given_date):
        todays_date = date.today()

        total_value = 0
        for ex in ex_list:
            total_value += self.get_value_date(ex, todays_date)
        return total_value
        
    def todays_value(self, ex_type):
        todays_date = date.today()
        return self.get_value_date(ex_type, todays_date)
    
    def todays_decomposition(self, ex_list): 
        ex_decom = {}
        for ex in ex_list:
            ex_decom[ex] = self.todays_value(ex)
        
        total = sum([ex_decom[k] for k in ex_decom.keys()])
        
        per_ex_decom = {}
        for ex in ex_list:
            per_nutri_decom[nutri] = 100.0 * (nutri_decom[nutri]/total)
        
        return per_nutri_decom
    
    def get_values_multiple_days(self, ex_type, ex_list, duration):
        todays_date = date.today()
        active_time_list = {}
        for i in range(duration):
            dt = todays_date - timedelta(days=0)
            if ex_list is None:
                active_time_list[str(dt)] = self.get_value_date(ex_type, dt)
            else:
                active_time_list[str(dt)] = self.get_total_value_date(ex_type, dt)
        return weeks_nutrients

    
    # Returns a dictionary whose keys are the dates in string    
    def last_weeks_values(self, ex_type, ex_list=None):
        return self.get_values_multiple_days(ex_type, ex_list, 7)
        
    # Returns a dictionary whose keys are the dates in string
    def last_months_values(self, ex_type, ex_list=None):
        return self.get_values_multiple_days(ex_type, ex_list, 28)    
        
    def get_calories_burnt_from_API(self, query, upc):
        calories_burnt = get_calories_burnt_information(query)
        return calories_burnt
    
    # Input dict should have the following keys:
    # 1)exercise_type
    # 2)active_time
    def insert_in_database(self, input_dict):
        data_dict = input_dict['manual_dict']
        calories_burnt = get_calories_burnt_from_API(input_dict['exercise_type'])
        data_dict['date_time'] = str(datetime.now())
        date_dict['calories_burnt'] = calories_burnt

        self.__database.insert_Row(self.__table_name,data_dict)
        return True
    
    # NEEDS TO BE DISCUSSED.
    # Goal dict should have the following keys:
    # One for each exercise type active time
    # and one for the total active time
    def set_goals(self, goal_dict):
        goal_dict['date_time'] = str(datetime.now())
        self.__database.insert_Row(self.__goal_table_name, goal_dict)
        
    def check_goal(self, key, ex_list):
        todays_date = date.today()
        todays_value = self.get_total_value_date(ex_list, todays_date)
        c = self.__database.get_Cursor()
        query = f"SELECT {key} FROM {self.__goal_table_name}\
                ORDER BY ID DESC LIMIT 1"

        try:
            c.execute(query)
            records = c.fetchall()
            goal_value = records[0]
            if todays_value >= goal_value:
                return False
            else:
                return True
        except:
            False

Column Names for Sleep Table in Database:
- 1) bed_date_time (DATETIME)
- 2) wakeup_date_time (DATETIME)

In [26]:
class Sleep(PersonalHealth):
    #TBD

In [ ]:
import pymysql

class DB:
    def __init__(self, host, user, password, db):
        self._host = host
        self._user = user
        self._password = password
        self._db = db

    def get_connection(self):
        connection = pymysql.connect(host=self._host, user=self._user, password=self._password, db=self._db)
        return connection

    def close_connection(self, connection):
        connection.close()

    def select_data(self, query):
        connection = self.get_connection()
        cursor = connection.cursor(pymysql.cursors.DictCursor)
        cursor.execute(query)
        data = cursor.fetchall()
        self.close_connection(connection)
        return data

    def insert_data(self, cmd):
        connection = self.get_connection()
        cursor = connection.cursor()
        cursor.execute(cmd)
        connection.commit()
        self.close_connection(connection)

        user1: diet carbs: 1, calories: 1 sugar: 1
        user1: sleep wakeup: 0, sleep: 1
        
    def insert_row(self, table, data_dict):
        keys = ""
        vals = ""
        for k, v in data_dict.items():
            keys += k + ', '
            vals += v + ', '
        keys = keys[:-2]
        vals = vals[:-2]
        cmd = f"insert into {table} ({keys}) values ({vals});"
        self.insert_data(cmd)

    def sel_time_frame(self, table, start_date, end_date, userID, params="*"):
        query = f"select {params} from {table} "
        query += f"join Users on Users.id={table}.UserID "
        query += f"where date >= {start_date} and date <= {end_date};"
        data = self.select_data(query)
        return data

    def get_sleep_minutes(self, start_date, end_date, userID):
        data = self.sel_time_frame("Sleep", start_date, end_date, userID, params="Minutes")
        return data

    def get_goals(self, userID):
        query = f"select * from Goals join Users on Users.id={table}.UserID;"
        data = self.sel_data(query)
        return data

